In [3]:
import pandas as pd
#from google.colab import files
#uploaded = files.upload()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
df=pd.read_excel('/content/drive/MyDrive/Munna_Research/Experiment/ecdata_new.xlsx')
df

,shop,product_name,comments,quality,catagories
0,daraz,MyCell Mi10 PLUS - 2.4 Feature Phone,Recommended but phone battery cover is not as ...,bad,a
1,daraz,Mini Hand Sanitizer Anti-Bacterial Hand Gel 30...,thanks a lot but what is the problem you guys ...,bad,a
2,daraz,"Vietnam hybrids coconut Tree , ১ পিস গাছের চারা",মাত্র ১১ দিনে ডেলিভারি পেয়েছি সে জন্য দারাজকে ...,bad,a
3,daraz,Bogesi Men Gold Artificial Leather Wallet,bogesi অরজিনাল প্রোডাক্টই পেয়েছি। ধন্যবাদ সেলর,bad,a
4,daraz,Craft HD LED TV - 32(Inch),Avarege...I Faced Some Problems But Seller And...,bad,a
...,...,...,...,...,...
5104,daraz,Black and White Cotton Un-Stitched Salwar Kame...,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,good,w
5105,daraz,Black and White Cotton Un-Stitched Salwar Kame...,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,good,w
5106,daraz,Both Sided Gum Tape - 2 inch - White\n \n \n...,orginal prodauct and draz delevery good prodau...,good,w
5107,daraz,2020 NEW STYLE Cotton Long Sleeve Formal Shirt...,Order r real shirt check koren\n\n\n\n\npleaz ...,good,w


In [46]:
#df=df.drop(['product_name','shop','quality'],axis=1)
df=df.drop(['product_name','shop','catagories'],axis=1)

In [50]:
#df.catagories.unique()
df.quality.unique()

array(['bad', 'good'], dtype=object)

In [49]:
#df['catagories'].replace({'c':'complain','w':'wrong delevery','a':'appreciation','r':'recommended'}, inplace=True)
df['quality'].replace({'bad ':'bad'}, inplace=True)
df.head()

,comments,quality
0,Recommended but phone battery cover is not as ...,bad
1,thanks a lot but what is the problem you guys ...,bad
2,মাত্র ১১ দিনে ডেলিভারি পেয়েছি সে জন্য দারাজকে ...,bad
3,bogesi অরজিনাল প্রোডাক্টই পেয়েছি। ধন্যবাদ সেলর,bad
4,Avarege...I Faced Some Problems But Seller And...,bad


In [8]:
df.catagories.unique()

array(['appreciation', 'complain', 'recommended', 'wrong delevery'],
      dtype=object)

In [51]:
df=df.dropna()
df

,comments,quality
0,Recommended but phone battery cover is not as ...,bad
1,thanks a lot but what is the problem you guys ...,bad
2,মাত্র ১১ দিনে ডেলিভারি পেয়েছি সে জন্য দারাজকে ...,bad
3,bogesi অরজিনাল প্রোডাক্টই পেয়েছি। ধন্যবাদ সেলর,bad
4,Avarege...I Faced Some Problems But Seller And...,bad
...,...,...
5104,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,good
5105,জামাটা ঠিক আছে কিন্তু অলনা টা মিলে নাই ইমেজে দ...,good
5106,orginal prodauct and draz delevery good prodau...,good
5107,Order r real shirt check koren\n\n\n\n\npleaz ...,good


In [56]:
from nltk.stem.porter import PorterStemmer

porter=PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [55]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    return text
df['comments']=df['comments'].apply(preprocessor)

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(strip_accents= None,
                    lowercase=False,
                    preprocessor=None,
                    tokenizer=tokenizer_porter,
                    norm='l2',
                    smooth_idf=True )
#y=df.catagories.values
y=df.quality.values
X=tfidf.fit_transform(df.comments)


In [58]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state= 44, test_size=0.2)
                                                   

LOGISTIC REGRATION



In [59]:
from sklearn.linear_model import LogisticRegression
lc = LogisticRegression()
lc.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [60]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
pred=lc.predict(X_test)

In [61]:
print(classification_report(y_test,pred))

              precision    recall  f1-score   support

         bad       0.85      0.89      0.87       446
        good       0.91      0.88      0.89       576

    accuracy                           0.88      1022
   macro avg       0.88      0.88      0.88      1022
weighted avg       0.89      0.88      0.88      1022



In [62]:
mat = confusion_matrix(pred, y_test)
pd.DataFrame(mat)

,0,1
0,398,71
1,48,505


KNN


In [63]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=6)
neigh.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')

In [64]:
pred=neigh.predict(X_test)
print(classification_report(y_test,pred))
mat = confusion_matrix(pred, y_test)
pd.DataFrame(mat)

              precision    recall  f1-score   support

         bad       0.81      0.81      0.81       446
        good       0.85      0.85      0.85       576

    accuracy                           0.83      1022
   macro avg       0.83      0.83      0.83      1022
weighted avg       0.83      0.83      0.83      1022



,0,1
0,361,87
1,85,489
